# Analisando transformadas com auto-valores e auto-vetores

Este capítulo está ligado ao seguintes objetivos didáticos do curos:
1. Interpretar e analisar transformações lineares
1. Aplicar a decomposição em valores singulares

Referências bibliográficas:

[Jim Hefferon - Linear Algebra - 4th Edition](https://hefferon.net/linearalgebra/) - Chap. Five, II.3

[Philip N. Klein - Coding the Matrix - 1st Edition](https://codingthematrix.com/) - Chap. 11-12

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
#import cv2 as cv
import pandas as pd
#import pygame

# Capítulo 5

# Parte 1

# Exercício 1
**Objetivo: Executar um sistema dinâmico e procurar por pontos de estabilidade**


No [capítulo 2](02-sistemas_lineares.ipynb), usamos uma matriz para representar dois sistemas que operam dinamicamente: a população de carcarás e sapos numa região fictícia, e a dinâmica de navegação de um web-surfista aleatório. Se precisar, revise esse material!

Esses dois sistemas são chamados de *sistemas dinâmicos* porque eles estão tratando de uma evolução de elementos ao longo do tempo. Como já falamos de carcarás e navegadores, vamos tratar agora de um outro sistema bem interessante, que é o de bicicletas públicas de Montreal.

Já por perto de 2010 a 2011, Montreal tinha um sistema de bicicletas públicas bastante parecido com as bicicletas Itaú que temos hoje em São Paulo. Só que Montreal é uma cidade que fica em uma montanha (cujo ponto mais alto é o [Mount Royal](https://www.google.com/maps/place/Mount+Royal+Park/@45.5023053,-73.6098349,14.25z/data=!4m13!1m7!3m6!1s0x4cc91a541c64b70d:0x654e3138211fefef!2sMontreal,+QC,+Canada!3b1!8m2!3d45.5018869!4d-73.5673919!3m4!1s0x4cc91a3b89d50ee1:0x4c8dc463a4718c9a!8m2!3d45.5015664!4d-73.5932179)), o que levou a um problema interessante: as pessoas tendem a tomar uma bicicleta emprestada na estação que fica na região mais alta e devolver em uma das estações das regiões mais baixas, mas raramente alguém faz o caminho inverso. O fim dessa história é que a [prefeitura contratou caminhões para ficarem levando bicicletas morro acima](https://www.csmonitor.com/World/Global-News/2011/0921/Montreal-s-public-bike-system-faces-uphill-battle), e uma consequência inesperada é que essa história seria um exemplo numa aula de álgebra linear dez anos depois num outro hemisfério.

---

Em nosso modelo, vamos fazer várias suposições que cabem para o nosso exemplo, mas que não necessariamente correspondem a dados reais.

* Vamos supor que temos três estações. As duas primeiras (E1 e E2) ficam na parte baixa da cidade. A outra estação (E3) fica na parte alta.
* Quando uma bicicleta é tomada emprestada em E1, ela tem 90% de chance de ser devolvida em E2 e 10% de chance de ser devolvida em E3.
* Quando uma bicicleta é tomada emprestada em E2, ela tem 95% de chance de ser devolvida em E1 e 5% de chance de ser devolvida em E3.
* Quando uma bicicleta é tomada emprestada em E3, ela tem 70% de chance de ser devolvida em E1 e 30% de chance de ser devolvida em E2.

Vamos supor que cada estação começa com 10 bicicletas, e que não há um limite máximo de bicicletas que podem ficar em cada estação.

Gostaríamos de saber:

(a) Após alguns dias de iterações aleatórias, quantas bicicletas esperamos encontrar em cada uma das estações?

(b) Com esses dados inventados, vai ser necessário contratar caminhões para levar bicicletas de alguma estação para outra? Para quais?

(c) Se, ao invés de começarmos nossas iterações com 10 bicicletas em cada estação, começarmos com todas as 30 bicicletas na estação E3, no topo do morro, devemos encontrar uma quantidade final de bicicletas diferente, ao fim de várias iterações?

---

In [4]:
import numpy as np
x = np.array([[0], [0], [30]])
A = np.array( [ [0, 0.9, 0.1], [0.95, 0, 0.05], [0.7, 0.3, 0]]).T
for _ in range(100):
    x = A @ x

print(x)
# um passo: x = A @ x
# implemente 100 passos seguidos!

# Número de bicicletas por estação = # total de bicicletas * P(bicicleta por estação)
print(x)

[[14.3359374]
 [13.5519198]
 [ 2.1121428]]
[[14.3359374]
 [13.5519198]
 [ 2.1121428]]



Discussão sobre a solução:

Sob um ponto de vista matemático, um ciclista aleatório parece se comportar de uma maneira muito parecida com um web-surfista aleatório! Veja o exercício 14 do capítulo de sistemas lineares para uma explicação sobre isso. Podemos representar nosso vetor de bicicletas como:

$$
x_0 = 30 \begin{bmatrix} 1/3 \\ 1/3 \\ 1/3 \end{bmatrix}
$$

e a matriz que representa as transições entre estações será a matriz $A$ de forma que $a[i,j]=P(s_t = i | s_{t-1}=j)$, isto é, a $a_{i,j}$ é a probabilidade de uma bicicleta ser deixada na estação $i$ sabendo que ela estava estação $j$:

$$
A = \begin{bmatrix} 
    0 & 0.9 & 0.7 \\
    0.95 & 0 & 0.3 \\
    0.05 & 0.1 & 0 
    \end{bmatrix}
$$



In [ ]:
import numpy as np
# Comecei com as matrizes já digitadas no código 
x = np.array([[1], [1], [1]])/3
A = np.array( [ [0, 0.95, 0.05], [0.9, 0, 0.1], [0.7, 0.3, 0]]).T
# um passo: x = A @ x
# implemente 100 passos seguidos!
for _ in range(1000):
    x = A@x

# Número de bicicletas por estação = # total de bicicletas * P(bicicleta por estação)
print(x)

In [ ]:
A@x

# Exercício 2
**Objetivo: calcular auto-valores e auto-vetores usando pacotes Python**

O problema das bicicletas de Montreal (assim como o PageRank e o problema dos carcarás do Capítulo 2) está ligado a uma característica de matrizes que é a existência de vetores e valores chamados de auto-valores e de auto-vetores. Quando um auto-vetor de uma matriz é multiplicado pela própria matriz, o resultado é um múltiplo do próprio vetor, isto é:

$$
Ax = x \lambda,
$$
onde: $A$ é a matriz, $x$ é o auto-vetor e $\lambda$ é o auto-valor correspondente.

Por exemplo:

$$
\begin{bmatrix} 
    2 & 0  \\
    0 & 1 
\end{bmatrix}
\begin{bmatrix} 
    1 \\
    0  
\end{bmatrix}
= 
\begin{bmatrix} 
    2 \\
    0  
\end{bmatrix}
=
2
\begin{bmatrix} 
    1 \\
    0  
\end{bmatrix},
$$
portanto $[1,0]^T$ é um auto-vetor de $[[2, 0],[0,1]]$, e $2$ é o auto-valor correspondente.

Se tivermos algum vetor na mesma direção de $x$ mas multiplicado por algum número real $\alpha$, isto é, $y=\alpha x$, podemos manter a mesma equação:

$$
A \alpha x = \alpha x \lambda \rightarrow Ay = y \lambda.
$$

a) No problema das bicicletas de Montreal, encontramos um auto-vetor da matriz $A$. Qual é esse auto-vetor, e qual é o auto-valor correspondente? 

Lembrando que:
$$
A = \begin{bmatrix} 
    0 & 0.9 & 0.7 \\
    0.95 & 0 & 0.3 \\
    0.05 & 0.1 & 0 
    \end{bmatrix}
$$

b) A instrução `w, v = np.linalg.eig(A)` retorna os auto-vetores da matriz A na variável `v` e os auto-valores correspondentes na variável `w`. Use essa função para calcular os auto-valores e auto-vetores da matriz $A$ do problema das bicicletas de Montreal.


In [6]:
A = np.array( [ [0, 0.9, 0.1], [0.95, 0, 0.05], [0.7, 0.3, 0]]).T
x = np.array([[1], [1], [1]])/3
for _ in range(1000):
    x = A @ x
print(x)

print("----")
w, v = np.linalg.eig(A)
print(w)
print(v)
print("Auto-vetor com maior auto-valor, normalizado:")
print( v[:,0]/v[0,0] )
print("Resultado da minha simulação, normalizado:")
print( x / x[0,0])

[[0.47815534]
 [0.45145631]
 [0.07038835]]
----
[ 1.         -0.93588989 -0.06411011]
[[-0.72298568 -0.72672227 -0.22345875]
 [-0.68261592  0.68570576 -0.56838065]
 [-0.10642936  0.04101651  0.7918394 ]]
Auto-vetor com maior auto-valor, normalizado:
[1.         0.94416244 0.14720812]
Resultado da minha simulação, normalizado:
[[1.        ]
 [0.94416244]
 [0.14720812]]


In [29]:
w, v = np.linalg.eig(A)
L = np.diag(w)
print(L)
A2 = v @ L @ np.linalg.inv(v)
A2.round(2)

x = np.array([[1], [1], [1]])/3
for _ in range(1000):
    x = A @ x
print("...")
x
v[:,0].reshape(-1,1)
alpha = (x / v[:,0].reshape(-1,1))[0,0]
print(v[:,0].reshape(-1,1)*alpha, x)

[[ 1.          0.          0.        ]
 [ 0.         -0.93588989  0.        ]
 [ 0.          0.         -0.06411011]]
...
[[0.47815534]
 [0.45145631]
 [0.07038835]] [[0.47815534]
 [0.45145631]
 [0.07038835]]


In [54]:
A = np.array([[0.25, 0.2], [0.75, 1.5]])
lambdas, v = np.linalg.eig(A)
print(lambdas)
x = np.random.rand(2,1)
for _ in range(200):
    x = A @ x
print(x / np.sum(x))

[0.13972794 1.61027206]
[[0.12818277]
 [0.87181723]]


# Exercício 3
**Objetivo: auto-vetores e auto-valores como pontos de estabilidade, expansão, ou colapso**

 Vamos retomar o problema da população de carcarás e de sapos no Capítulo 2. Nesse problema, tínhamos a situação em que carcarás estão no mesmo habitat que sapos, obedecendo às seguintes regras:

* A população de carcarás naturalmente cai em 20% a cada mês
* Para cada cinco mil sapos existentes naquele mês, um novo carcará nasce
* A população de sapos naturalmente sobe em 10% porque eles se reproduzem
* A cada mês, cada carcará consegue comer 100 sapos

Isso significa que podemos escrever equações para calcular o número de carcarás e de sapos a cada mês, tomando por base a quantidade deles no mês anterior:

$$ 
\begin{cases}
    \begin{aligned}
    c_t & = 0.8 c_{t-1} + 0.2 s_{t-1} \\
    s_t & = - 0.1 c_{t-1} + 1.1 s_{t-1} \\
    \end{aligned}
\end{cases}
$$

Podemos fazer esse cálculo usando a operação matricial:
$$
\begin{bmatrix}
    c_t \\
    s_t 
\end{bmatrix}
=
\begin{bmatrix}
    0.8 & 0.2\\
    -0.1 & 1.1
\end{bmatrix} 
\begin{bmatrix}
    c_{t-1} \\
    s_{t-1}
\end{bmatrix}
$$

Como já vimos, podemos calcular a dinâmica da população ao longo do tempo partindo, por exemplo, de 600 carcarás e 500 mil sapos:

In [ ]:
import matplotlib.pyplot as plt
A = np.array([[0.8, 0.2], [-0.1,  1.1]])
x = np.array([[600], [500]])
populacao_carcaras = []
populacao_sapos = []
for _ in range(1000):
    populacao_carcaras.append(x[0,0])
    populacao_sapos.append(x[1,0])
    x = A @ x

plt.figure(figsize=(7,2))
plt.plot(populacao_carcaras, label='Carcarás')
plt.plot(populacao_sapos, label='Sapos (* 1000)')
plt.ylim([0,800])
plt.ylabel('População')
plt.xlabel('Semanas')
plt.show()

a) Usando a função `np.linalg.eig`, encontre os auto-valores e auto-vetores da matriz que representa a dinâmica da população de carcarás e sapos. 


In [ ]:
w, v = np.linalg.eig(A)
print(w, v)


b) Qual é o auto-vetor correspondente ao auto-valor $1$?



c) Como esse auto-vetor corresponde à dinâmica de população que observamos ao simular o sistema?



In [ ]:
print(w)
print(v)
print( v[:,1]/v[0,1] )
print( x / x[0,0])



d) O que acontece com o sistema se as condições iniciais forem iguais ao auto-vetor correspondente ao outro auto-valor?



In [ ]:
x = v[:,0].reshape( (2,1))
x *= -100
print(x)
import matplotlib.pyplot as plt
A = np.array([[0.8, 0.2], [-0.1,  1.1]])
populacao_carcaras = []
populacao_sapos = []
for _ in range(100):
    populacao_carcaras.append(x[0,0])
    populacao_sapos.append(x[1,0])
    x = A @ x

plt.figure(figsize=(7,2))
plt.plot(populacao_carcaras, label='Carcarás')
plt.plot(populacao_sapos, label='Sapos (* 1000)')
#plt.ylim([0,800])
plt.ylabel('População')
plt.xlabel('Semanas')
plt.show()



e) Explique essa dinâmica usando o conceito de auto-valores e auto-vetores, isto é, a equação $Ax = x \lambda$ onde $A$ é a matriz da dinâmica, $x$ é o auto-vetor e $\lambda$ é o auto-valor correspondente.

# Exercício 4
*Objetivo: encontrar uma matriz à partir de seus auto-valores e auto-vetores*

Em algumas situações (por exemplo, quando estou montando essa lista de exercícios - mas, de forma mais geral, quando estamos projetando sistemas dinâmicos), gostaríamos de encontrar matrizes que têm os autovalores e autovetores que queremos. Para isso, vamos escrever a equação de autovetores e autovalores na forma matricial.

Quando estamos falando de somente um auto-vetor, podemos escrever:

$$
Ax = x \lambda
$$

Porém, se tivermos dois auto-vetores, e seus auto-valores correspondentes, temos na verdade um sistema:

$$
\begin{cases}
Ax_1 = x_1 \lambda_1 \\
Ax_2 = x_2 \lambda_2 
\end{cases}
$$

Esse sistema pode ser escrito na forma de uma multiplicação matricial, se assumirmos que nossos auto-vetores são vetores-coluna:

$$
A \begin{bmatrix} x_1 & x_2 \end{bmatrix} = \begin{bmatrix} x_1 & x_2 \end{bmatrix} \begin{bmatrix} \lambda_1 & 0 \\ 0 & \lambda_2 \end{bmatrix} 
$$

Multiplicando os dois lados da equação por $\begin{bmatrix} x_1 & x_2 \end{bmatrix}^{-1}$, ficamos com:

$$
A \begin{bmatrix} x_1 & x_2 \end{bmatrix}\begin{bmatrix} x_1 & x_2 \end{bmatrix}^{-1} = \begin{bmatrix} x_1 & x_2 \end{bmatrix} \begin{bmatrix} \lambda_1 & 0 \\ 0 & \lambda_2 \end{bmatrix} \begin{bmatrix} x_1 & x_2 \end{bmatrix}^{-1}
$$

e, portanto:
$$
A = \begin{bmatrix} x_1 & x_2 \end{bmatrix} \begin{bmatrix} \lambda_1 & 0 \\ 0 & \lambda_2 \end{bmatrix} \begin{bmatrix} x_1 & x_2 \end{bmatrix}^{-1}
$$

Por exemplo, se quiséssemos uma matriz cujos auto-valores são $0.7$ e $0.2$ e os auto-vetores correspondentes são $[1,0]^T$ e $[0.5, 0.5]^T$, então deveríamos calcular:

$$
A = \begin{bmatrix} 1 & 0.5 \\ 0 & 0.5 \end{bmatrix} \begin{bmatrix} 0.7 & 0 \\ 0  & 0.2 \end{bmatrix} \begin{bmatrix} 1 & 0.5 \\ 0 & 0.5 \end{bmatrix}^{-1}
$$

Encontre a matriz com autovetores $[1,0]^T$ e $[0.5, 0.5]^T$ e autovalores $0.7$ e $0.2$.

In [ ]:
S = np.array( [[1, 0], [0.5, 0.5]]).T
L = np.array( [ [0.7, 0], [0, 0.2] ])
S @ L @ np.linalg.inv(S)

In [ ]:
import matplotlib.pyplot as plt
S = np.array( [[1, 0.5], [1.5, 0.5]]).T
L = np.array( [ [1, 0], [0, 1] ])
A_din = S @ L @ np.linalg.inv(S)
print(A_din)
x = np.array([[1000, 1100]]).T
print(x)


In [ ]:

populacao_carcaras = []
populacao_sapos = []
for _ in range(100):
    populacao_carcaras.append(x[0,0])
    populacao_sapos.append(x[1,0])
    x = A_din @ x

plt.figure(figsize=(7,2))
plt.plot(populacao_carcaras, label='Carcarás')
plt.plot(populacao_sapos, label='Sapos (* 1000)')
#plt.ylim([0,800])
plt.ylabel('População')
plt.xlabel('Semanas')

plt.show()

# Exercício 5
*Objetivo: relacionar autovalores ao colapso, explosão e ao equilíbrio de sistemas dinâmicos*

Até o momento, usamos um vetor $v$ para representar o estado de sistemas - a quantidade de bicicletas nas estações em algum momento, a quantidade de animais em alguma semana, e assim por diante. Vamos chamar nossa condição inicial de $v_0$, significando o vetor de estados $v$ no tempo $0$ (e, portanto, $v_1$ é o estado após uma iteração, e $v_{223}$ é o estado após $223$ iterações).

Quando aplicamos uma matriz sobre um vetor de estados qualquer $v_0$, isto é, quando damos um passo em nosso sistema dinâmico, temos a situação:

$$
v_1 = A v_{0}
$$

Podemos aplicar novamente a matriz sobre $v_1$, encontrando:
$$
v_2 = A v_1 = A A v_0 = A^2 v_0.
$$

E, ao longo de $N$ iterações, teremos:
$$
v_N = A^N v_0
$$
---

Isso é  o que fizemos diversas vezes até este momento. Vamos agora re-escrever essa mesma equação na usando a forma de autovalores e autovetores. As passagens que seguem parecem um pouco longas porque as expressões são longas, mas são somente a consequência de trocar $A$ nas equações acima pela decomposição matricial:

$$
A = \begin{bmatrix} x_1 & x_2 \end{bmatrix} \begin{bmatrix} \lambda_1 & 0 \\ 0 & \lambda_2 \end{bmatrix} \begin{bmatrix} x_1 & x_2 \end{bmatrix}^{-1}
$$

Aplicando essa mudança na primeira expressão, temos:

$$
v_1 = A v_0 = \begin{bmatrix} x_1 & x_2 \end{bmatrix} \begin{bmatrix} \lambda_1 & 0 \\ 0 & \lambda_2 \end{bmatrix} \begin{bmatrix} x_1 & x_2 \end{bmatrix}^{-1} v_{0}
$$

Na segunda iteração, temos:

$$
v_2 = A v_1 = \left( A \right) A v_0 = \left( \begin{bmatrix} x_1 & x_2 \end{bmatrix} \begin{bmatrix} \lambda_1 & 0 \\ 0 & \lambda_2 \end{bmatrix} \begin{bmatrix} x_1 & x_2 \end{bmatrix}^{-1} \right) \begin{bmatrix} x_1 & x_2 \end{bmatrix} \begin{bmatrix} \lambda_1 & 0 \\ 0 & \lambda_2 \end{bmatrix} \begin{bmatrix} x_1 & x_2 \end{bmatrix}^{-1}v_0
$$

Veja que podemos simplificar a multiplicação $ \begin{bmatrix} x_1 & x_2 \end{bmatrix}^{-1} \begin{bmatrix} x_1 & x_2 \end{bmatrix}$ no meio da cadeia de multiplicações, já que ela é igual à identidade. Ficamos então com:

$$
v_2 = \begin{bmatrix} x_1 & x_2 \end{bmatrix} \begin{bmatrix} \lambda_1 & 0 \\ 0 & \lambda_2 \end{bmatrix} \begin{bmatrix} \lambda_1 & 0 \\ 0 & \lambda_2 \end{bmatrix} \begin{bmatrix} x_1 & x_2 \end{bmatrix}^{-1}v_0
$$

Podemos resumir a multiplicação matricial $\begin{bmatrix} \lambda_1 & 0 \\ 0 & \lambda_2 \end{bmatrix} \begin{bmatrix} \lambda_1 & 0 \\ 0 & \lambda_2 \end{bmatrix}$ como $\begin{bmatrix} \lambda_1 & 0 \\ 0 & \lambda_2 \end{bmatrix}^2$, ficando com:

$$
v_2 = \begin{bmatrix} x_1 & x_2 \end{bmatrix} \begin{bmatrix} \lambda_1 & 0 \\ 0 & \lambda_2 \end{bmatrix} ^2 \begin{bmatrix} x_1 & x_2 \end{bmatrix}^{-1}v_0
$$

Fazendo uma nova multiplicação por $A$, vamos encontrar:
$$
v_3 = \left( A \right) v_2 = \left( \begin{bmatrix} x_1 & x_2 \end{bmatrix} \begin{bmatrix} \lambda_1 & 0 \\ 0 & \lambda_2 \end{bmatrix} \begin{bmatrix} x_1 & x_2 \end{bmatrix}^{-1} \right) \begin{bmatrix} x_1 & x_2 \end{bmatrix} \begin{bmatrix} \lambda_1 & 0 \\ 0 & \lambda_2 \end{bmatrix} ^2 \begin{bmatrix} x_1 & x_2 \end{bmatrix}^{-1}v_0
$$

Novamente, podemos trocar a multiplicação matricial $ \begin{bmatrix} x_1 & x_2 \end{bmatrix}^{-1} \begin{bmatrix} x_1 & x_2 \end{bmatrix}$ pela identidade, e agrupar $\begin{bmatrix} \lambda_1 & 0 \\ 0 & \lambda_2 \end{bmatrix} \begin{bmatrix} \lambda_1 & 0 \\ 0 & \lambda_2 \end{bmatrix}^2$ como $\begin{bmatrix} \lambda_1 & 0 \\ 0 & \lambda_2 \end{bmatrix}^3$, ficando com:

$$
v_3 = \begin{bmatrix} x_1 & x_2 \end{bmatrix} \begin{bmatrix} \lambda_1 & 0 \\ 0 & \lambda_2 \end{bmatrix} ^3 \begin{bmatrix} x_1 & x_2 \end{bmatrix}^{-1} v_0
$$

Podemos fazer essa mesma operação $N$ vezes, ficando com:

$$
v_N = \begin{bmatrix} x_1 & x_2 \end{bmatrix} \begin{bmatrix} \lambda_1 & 0 \\ 0 & \lambda_2 \end{bmatrix} ^N \begin{bmatrix} x_1 & x_2 \end{bmatrix}^{-1} v_0
$$

Como $\begin{bmatrix} \lambda_1 & 0 \\ 0 & \lambda_2 \end{bmatrix} ^N = \begin{bmatrix} \lambda_1 ^N & 0 \\ 0 & \lambda_2 ^N \end{bmatrix}$, a expressão fica:

$$
v_N = \begin{bmatrix} x_1 & x_2 \end{bmatrix} \begin{bmatrix} \lambda_1^N & 0 \\ 0 & \lambda_2^N \end{bmatrix} \begin{bmatrix} x_1 & x_2 \end{bmatrix}^{-1} v_0
$$

---

Veja que agora sabemos que:
$$
A^N = \begin{bmatrix} x_1 & x_2 \end{bmatrix} \begin{bmatrix} \lambda_1^N & 0 \\ 0 & \lambda_2^N \end{bmatrix} \begin{bmatrix} x_1 & x_2 \end{bmatrix}^{-1}
$$ 

Usando valores à sua escolha e supondo que $A \in \mathbb{R}^{2\times 2}$, monte exemplos que mostrem o que acontece com os valores de $A^N$ se:

1. Todos os auto-valores forem positivos e menores que 1.
2. Somente um auto-valor for igual a 1, e os demais forem positivos e menores que 1.
3. Um dos auto-valores for maior que 1.
4. Todos os auto-valores forem iguais a 1.
5. Relacione as suas respostas anteriores aos conceitos de *explosão* (tender ao infinito ao longo de muitas iterações), *colapso* (tender a zero ao longo de muitas iterações) e *equilíbrio* (tender a um valor constante não-zero ao longo de muitas iteraçõees).

In [ ]:
S = np.array( [[1, 0], [0.5, 0.5]]).T # Provavelmente podemos resolver este exercício sem alterar isso.
L = np.array( [ [0.7, 0], [0, 0.2] ]) # Você provavelmente quer alterar esta linha!


# Exercício 6
*Objetivo: relacionar colapso, explosão e equilíbrio situações práticas*

Neste exercício, estamos buscando entender como podemos usar a equação que vimos acima:

$$
v_N = \begin{bmatrix} x_1 & x_2 \end{bmatrix} \begin{bmatrix} \lambda_1^N & 0 \\ 0 & \lambda_2^N \end{bmatrix} \begin{bmatrix} x_1 & x_2 \end{bmatrix}^{-1} v_0
$$

pode ser usada para encontrar comportamentos ao longo do tempo em sistemas dinâmicos.


(a) Usando auto-valores e auto-vetores no caso da população de carcarás, justifique matematicamente o fato de que muito provavelmente a população deve tender a um equilíbrio ao longo de muitos meses. 

In [ ]:
A = np.array([[0.8, 0.2], [-0.1,  1.1]])
w, v = np.linalg.eig(A)
print(w)
print(v)

(b) Partindo do caso da população de carcarás, suponha a seguinte situação. A população local decidiu que existem muitos carcarás nas redondezas, e por isso autorizou a caça. Com isso, ao fim do mês, ao invés de morrerem 20% dos carcarás existentes, morrem 80% dos carcarás existentes. Use auto-valores e auto-vetores para prever se esse processo de caça será eficaz para conter a população de carcarás. Após, confirme seu resultado usando uma simulação.

In [ ]:
A = np.array([[0.2, 0.2], [-0.1,  1.1]]) # Depois da caça, a nova matriz A ficou assim!!!

w, v = np.linalg.eig(A)
print(w)
print(v)
x = np.array([[1000, 1100]]).T

populacao_carcaras = []
populacao_sapos = []
for _ in range(100):
    populacao_carcaras.append(x[0,0])
    populacao_sapos.append(x[1,0])
    x = A @ x

plt.figure(figsize=(7,2))
plt.plot(populacao_carcaras, label='Carcarás')
plt.plot(populacao_sapos, label='Sapos (* 1000)')
#plt.ylim([0,800])
plt.ylabel('População')
plt.xlabel('Semanas')

plt.show()

(c) Após a publicação de imagens chocantes sobre a caça dos carcarás, foram aprovadas leis de proteção e cuidado ambiental que fazem com que a mortalidade dos carcarás de um mês para o outro seja de apenas 19%. Usando auto-valores e auto-vetores, determine qual é o efeito dessa lei, a longo prazo, para as populações de carcarás e sapos na região.

In [ ]:
A = np.array([[0.81, 0.2], [-0.1,  1.1]]) # Depois da nova legislação, a nova matriz A ficou assim!!!


w, v = np.linalg.eig(A)
print(w)
print(v)

x = np.array([[1000, 1100]]).T

populacao_carcaras = []
populacao_sapos = []
for _ in range(1000):
    populacao_carcaras.append(x[0,0])
    populacao_sapos.append(x[1,0])
    x = A @ x

plt.figure(figsize=(7,2))
plt.plot(populacao_carcaras, label='Carcarás')
plt.plot(populacao_sapos, label='Sapos (* 1000)')
#plt.ylim([0,800])
plt.ylabel('População')
plt.xlabel('Semanas')

plt.show()

# Parte 2

# Exercício 7
*Objetivo: representar dados numéricos em uma matriz*

Quando vamos processar uma grande quantidade de dados, podemos representá-los usando, por exemplo, dataframes de Pandas:

In [ ]:
import pandas as pd
import numpy as np
col1 = np.random.randn(5)
col2 = np.random.randn(5)
df = pd.DataFrame()
df['col1'] = col1
df['col2'] = col2
print(df.head())

Podemos passar nossos dados para um array de Numpy usando:

In [ ]:
data = df[['col1', 'col2']].to_numpy()
print(data)

Neste exercício, temos o objetivo de usar um array de numpy para substituir o dataframe. Usaremos o conjunto de dados [Automobile Data](https://www.kaggle.com/datasets/toramky/automobile-dataset), que é uma compilação de dados do *1985 Ward's Automotive Yearbook* e de informações fornecidas por companhias de seguro da época. Para este exercício, estamos especialmente interessados nas *features* *curb-weight* (o peso total de um veículo com o tanque cheio, sem contar os passageiros) e *highway-mpg* (o consumo de combustível do veículo, em milhas por galão).

(a) Modifique o código abaixo para que as duas colunas com dados fiquem em um numpy array `X` de dimensão `N x 2`, onde `N` é o número de veículos do dataset;

(b) Modifique o código abaixo para que a chamada `plt.scatter` use o array `X` ao invés de chamar o dataframe diretamente;

(c) Quais informações foram perdidas quando passamos a usar arrays de numpy no lugar de um dataframe?

In [ ]:
import matplotlib.pyplot as plt

df = pd.read_csv('Automobile_data.csv')
features = ["highway-mpg", "curb-weight"]
df = df[features]

plt.figure()
plt.scatter(df['highway-mpg'], df['curb-weight'])
plt.show()

# Exercício 8

*Objetivo: relacionar a equação da matriz de covariância a seu código computacional*

Lembre-se que a covariância entre duas colunas $x$ e $y$ de um `dataframe` é calculada por:

$$
C = \frac{\sum_{n=0}^{N-1} (x_n-\bar{x})(y_n-\bar{y})}{N-1}
$$

Podemos usar Pandas imediatamente para calcular a covariância entre todas as colunas de um dataframe usando:

In [ ]:
df.cov()

Neste exercício, vamos implementar essa mesma operação em Numpy. Para isso, use o array $X$ que você criou no exercício anterior. Uma das opções é usar o procedimento abaixo:

(a) Centralizar cada coluna, isto é, subtrair, de cada coluna, seu valor médio (implementando as operações $x_n-\bar{x}$ e $y_n-\bar{y}$)

(b) Multiplicar as colunas elemento a elemento (usando o operador `*`)

(c) Somar o resultado da multiplicação ($np.sum$)

(d) Dividir o resultado por $N-1$.

Outra opção é usar a biblioteca `numpy` para calcular a matriz de covariância:

In [ ]:
np.set_printoptions(suppress = True) #impede a notação científica

X = df.to_numpy()
y = np.mean(X, axis=0, keepdims=True)
X = X - np.mean(X, axis=0, keepdims=True) # Linha 2
C = X.T @ X # Linha 3
C /= X.shape[0]-1 # Linha 4

print(C)
np.cov(X.T)

Relacione cada uma das linhas do código acima a uma parte da equação da covariância.

# Exercício 9
*Objetivo: relacionar os autovetores da matriz de covariância à direção de espalhamento dos dados*

Os autovalores e autovetores da matriz de covariância podem ser calculados por:

In [ ]:
w, v = np.linalg.eig(C)
print(w, v)
v0 = v[:,0]
v1 = v[:,1]

(a) Qual dos autovetores de $C$ está ligado a um autovalor mais alto?

(b) Use como base o código abaixo. Adicione nele uma reta na direção do autovetor que você escolheu no ítem (a). Você está buscando um resultado parecido com:

<img src="reta_pca_1.png"></img>

(c) O que a reta desenhada no ítem (b) indica sobre os seus dados?

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

X = df.to_numpy()
X = X - np.mean(X, axis=0, keepdims=True) # Linha 2
C = X.T @ X # Linha 3
C /= X.shape[0]-1 # Linha 
w, v = np.linalg.eig(C)

plt.figure()
plt.scatter(X[:,0], X[:,1])
plt.xlim(-18,25)
plt.ylim(-1200,1700)
plt.show()

# Exercício 10
*Objetivo: fazer uma projeção PCA como uma mudança de bases*

No exercício anterior, vimos que nossos pontos, embora tenham duas dimensões, podem ser razoavelmente bem explicados usando somente uma reta. Então, se pudermos projetar nossos pontos sobre essa reta, podemos reduzir o número de dimensões (ou: reduzir o número de *features*) relacionado a cada ponto.

Lembre-se que uma *base* significa que passamos a representar nossos pontos como a combinação linear de vetores-base. Se a base tem vetores-linha $v_1$ e $v_2$, então nossos pontos originais $X$ podem ser representados como pontos $Y$ se:

$$
 Y [v_1, v_2]^T= X
$$

Lembre-se que, neste caso, $X$ e $Y$ devem ter um ponto por linha e uma feature por coluna.

Então, podemos encontrar $Y$ usando:
$$
 Y = X ([v_1, v_2]^T) ^{-1}
$$

À partir do código abaixo, obtenha a matriz $Y$ e escolha somente a coluna correspondente à reta que você traçou no exercício anterior. Mostre, na figura abaixo, onde os pontos "reduzidos" de $Y$ devem aparecer na reta. Você deve encontrar algo parecido com:

<img src="projecao_pca_1.png"></img>

In [ ]:
X = df.to_numpy()
X = X - np.mean(X, axis=0, keepdims=True)
Y = X@np.linalg.inv(v.T)
#Y = X@v
#print(Y)
plt.figure(figsize=(3,3))

plt.scatter(X[:,0], X[:,1], c='b', label='Pontos originais')
plt.scatter(y[:,1]*v[0,1], y[:,1]*v[1,1], c='r', label='Pontos projetados')
plt.xlim(-18,25)
plt.ylim(-1200,1700)
plt.show()

# Parte 3

Nesta parte, vamos usar a ideia de projetar elementos em um espaço de dimensão mais baixa para visualizar dados.

Para isso, precisamos de um resultado teórico, que é o seguinte:

A matriz de auto-vetores $P$ da matriz de covariância $C$ é ortonormal, isto é:

$$
P^{-1} = P^T
$$

A demonstração disso é opcional, embora interessante, e está num [arquivo separado](demonstracao_ortonormalidade.md).

# Exercício 11
**Objetivo: fazer uma projeção PCA para visualizar dados de alta dimensão**

Existe um problema que encontramos em muitos conjuntos de dados, que é o fato de eles terem *muitas* features. Quando isso acontece, é difícil visualizar os nossos pontos. Uma das possíveis soluções para isso é encontrar os auto-vetores e auto-valores da matriz de covariância dos dados, e então usar somente os dois auto-vetores ligados aos maiores auto-valores para projetar nossos dados. Desta forma, temos uma projeção que permite a visualização dos dados levando à pouca distorção de suas posições.

Este procedimento se chama Análise de Componentes Principais, ou Principal Component Analysis (PCA). Trata-se de um algoritmo bem conhecido, que está executado abaixo.

Partindo do código, responda:

a) Quantas features existem na variável `X` originalmente?

b) Qual é a dimensão da matriz de covariância `C`?

c) Qual é a linha que seleciona os auto-vetores relacionados aos maiores auto-valores?

d) Qual é a linha que projeta as features `X` sobre a base formada pelos dois auto-vetores escolhidos?

e) Por que a forma $Y = XV$ é equivalente a $Y = X(V^{-1})^T$? Dica: use a demonstração acima!

f) Quantas features há em $Y$?

In [ ]:
a = np.array([5,4,3,2])
np.argsort(a)

In [ ]:
# Carregando dados novamente
df = pd.read_csv('Automobile_data.csv').select_dtypes(['number'])
X = df.to_numpy()


# Calculando matriz de covariância
X = X - np.mean(X, axis=0, keepdims=True)
C = X.T @ X
C /= X.shape[0]-1
w, v = np.linalg.eig(C)

w_ = np.argsort(w)

V = v [ :, [w_[-1], w_[-2]]]
Y = X @ V

plt.figure()
plt.scatter(Y[:,0], Y[:,1])
plt.show()


# Exercício 12
**Objetivo: usar a biblioteca sklearn para realizar uma projeção PCA**

O código abaixo exemplifica o uso da biblioteca `sklearn` para realizar uma projeção PCA equivalente à que fizemos acima.

a) Observando as figuras geradas, você diria que os códigos fazem procedimentos equivalentes?

b) A variável `pca.components_` mostra os componentes principais encontrados pela projeção. Como ela se relaciona à matriz `V` do exercício acima?

c) A variável `pca.explained_variance_ratio` mostra o quanto da variância dos dados está explicada por cada componente da projeção. Verificando os resultados desta projeção, você diria que é seguro assumir que os dados estão praticamente posicionados sobre uma reta?

In [ ]:
from sklearn.decomposition import PCA

df = pd.read_csv('Automobile_data.csv').select_dtypes(['number'])
X = df.to_numpy()
pca = PCA(2)
Y = pca.fit_transform(X)

plt.figure()
plt.scatter(Y[:,0], Y[:,1])
plt.show()

print(pca.components_)
print(pca.explained_variance_ratio_)

# Parte 4

# Exercício 13
**Objetivo: ligar a decomposição por auto-valores e auto-vetores ao seu respectivo código computacional e verificar sua limitação para matrizes quadradas**

Já sabemos que uma matriz quadrada $X$ pode ser decomposta em termos de seus auto-vetores (na matriz $P$) e auto-valores (na diagonal da matriz $D$) na forma:

$$
A = P D P^{-1}.
$$

O código abaixo realiza essa decomposição e a recomposição.

a) Por que a matriz `B` no código é igual à matriz `A`?

b) Modifique o código para que a matriz `A` deixe de ser quadrada (por exemplo, adicionando uma linha ou coluna). Que erro é apontado no código?

c) Se a matriz $A$ na equação $A = P D P^{-1}$ não for quadrada, que problema deve aparecer na operação $P D P^{-1}$ em relação à dimensões de $P$ e $D$?


In [ ]:
A = np.array( [[1, 2], [2, 3]] )
w, v = np.linalg.eig(A)
#print(w, v)
w_ = np.diag(w)
#print(w_)
B = v @ w_ @ np.linalg.inv(v)

print(B)

# Exercício 14
**Objetivo: entender SVD como uma decomposição**

Quando calculamos $P$ e $D$ em $A=PDP^{-1}$, estamos *decompondo* a matriz $A$ em *componentes* ($P$, $D$ e $P^{-1}$).

Essa decomposição não é a única possibilidade. Uma outra, muito utilizada, é a decomposição em valores singulares (*Singular Value Decomposition*, ou **SVD**):

$$
A = U \Sigma V^T
$$

No código abaixo:

a) Confirme que a matriz `B` é igual à matriz `A`

b) Compare o conteúdo das matrizes `u`, `sigma` e `vt` ao conteúdo das matrizes `v` e `w_` no exercício anterior.

c) Altere o código para verificar se a decomposição SVD ainda funciona se a matriz `A` não for quadrada.

d) Se a matriz `A` não é quadrada, quais são as dimensões das matrizes `u`, `sigma` e `vt`?

e) O que a função `diagsvd` faz, e por que ela tem três parâmetros?

In [ ]:
from scipy.linalg import svd, diagsvd
import numpy as np
A = np.array( [[1, 2], [2, 3]] )
u, s, vt = svd(A)
sigma = diagsvd(s, A.shape[0], A.shape[1])

B = u @ sigma @ vt
print(A)
print(B)

# Exercício 15
**Objetivo: comparar SVD ao PCA**

Na decomposição SVD, usamos a formulação:

$
A = U \Sigma V^T,
$

onde:

* As colunas de $U$ são os auto-vetores de $A^T A$,
* As colunas de $V$ (e, portanto, as linhas de $V^T$) são auto-vetores de $A A^T$,
* $\Sigma$ é uma matriz onde $s_{i,i}$ é a raiz quadrada dos auto-valores de $A^T A$ ou de $A A^T$.

A [demonstração disso está aqui](./demonstracao_svd.md), e uma outra perspectiva sobre essa demonstração está [aqui](https://gregorygundersen.com/blog/2018/12/20/svd-proof/).


---

Na decomposição PCA, calculamos a matriz de covariância $C = A A^T$ e fazemos a projeção:

$
C = P D P^T,
$

onde:

* As colunas de $P$ são os auto-vetores de $C=A A^T$,
* A diagonal principal de $D$ tem os auto-valores de $C$.

---

Lembrando-se disso:

a) Como poderíamos usar a decomposição SVD para encontrar os componentes principais de $A$? 

b) Seria correto dizer que: "a decomposição SVD e a decomposição PCA são a mesma decomposição"?

# Exercício 16
**Objetivo: Manipular a matriz $\Sigma$ em uma imagem e verificar os resultados**

O código abaixo abre uma imagem em tons de cinza e faz uma decomposição SVD.

a) Qual é a dimensão da imagem lida?

b) Quais são as dimensões das matrizes `u`, `sigma` e `vt`?

c) Os valores da matriz `sigma` vêm da matriz `s`. O que acontece com a imagems se o primeiro elemento de `s` for modificado para ficar igual a `0` (dica: use `s[0] *= 0`)?

d) Remova progressivamente mais elementos da matriz `s` igualando-os a zero. O que acontece com a imagem formada?

e) Agora, remova elementos do *final* da matriz `s` igualando-os a zero. O que acontece com a imagem?

f) Experimente perceptualmente e verifique quantos elementos você consegue remover do final da matriz `s` até que perceba alguma diferença na imagem?

In [ ]:
from scipy.linalg import svd, diagsvd
import matplotlib.image as mpimg

A = mpimg.imread("rabbit.jpeg").mean(axis=2)[0::4,0::4]

u, s, vt = svd(A)

# Altere daqui para baixo
s = s
s[-100:] *= 0.0
# Altere daqui para cima

sigma = diagsvd(s, A.shape[0], A.shape[1])

B = u @ sigma @ vt

plt.subplot(1,2,1)
plt.title('Imagem original')
plt.imshow(A, cmap='gray')
plt.subplot(1,2,2)
plt.title('Imagem alterada')
plt.imshow(B, cmap='gray')
plt.show()

# Exercício 17
**Objetivo: Remover ruídos de uma imagem manipulando a matriz S**

Uma possível maneira de remover ruídos de uma imagem é a seguinte:

* Fazemos a decomposiçao SVD da imagem
* Removemos os elementos de menor magnitude da matriz $\Sigma$
* Reconstituímos a imagem

a) Aplique esse procedimento para tentar remover os ruídos que foram adicionados à imagem abaixo.

b) Qual é a relação entre o ruído removido da imagem e a distorção causada na imagem?

In [ ]:
from scipy.linalg import svd, diagsvd
import matplotlib.image as mpimg

A = mpimg.imread("rabbit.jpeg").mean(axis=2)[0::4,0::4]
alpha = 0.3
A += alpha * np.random.random(A.shape) * np.max(A)

u, s, vt = svd(A)

# Altere daqui para baixo
s = s
s[-170:] *= 0
# Altere daqui para cima

sigma = diagsvd(s, A.shape[0], A.shape[1])

B = u @ sigma @ vt


plt.subplot(1,2,1)
plt.title('Imagem ruidosa')
plt.imshow(A, cmap='gray')
plt.subplot(1,2,2)
plt.title('Imagem com ruído removido')
plt.imshow(B, cmap='gray')
plt.show()

# Exercício 18
**Objetivo: Comprimir uma imagem modificando a matriz $\Sigma$**

Muitas vezes, admitimos perdas em um processo de compressão. Isso acontece, por exemplo, em compressão de imagens ou de sons: uma pequena distorção, quase imperceptível, é aceitável, desde que isso possibilite comprimir dados.

A compressão de dados funciona "apagando" elementos de uma imagem ou de um som. As diferentes estratégias de compressão se diferenciam em *como escolher* os elementos que são mais imperceptíveis dentro de uma imagem ou som.

Nesta atividade, vamos fazer um compressor de imagens baseado em SVD. O método aplicado é bem conhecido, e uma boa explicação dele pode ser encontrada [aqui](https://medium.com/balabit-unsupervised/image-compression-using-singular-value-decomposition-de20451c69a3).

Veja que, quando eliminamos valores de $\Sigma$ fazendo-os ficar igual a zero, isso é equivalente a remover alguns elementos das matrizes $U$ e de $V^T$. Por exemplo, na decomposição:

$$
U \Sigma V^T = 
\begin{bmatrix}
1 & 2 \\
3 & 4
\end{bmatrix}
\begin{bmatrix}
1 & 0 \\
0 & 0.2
\end{bmatrix}
\begin{bmatrix}
4 & 3 \\
2 & 1
\end{bmatrix}
$$

Se eliminamos o valor $0.2$ de $\Sigma$, ficamos com:

$$ 
\begin{bmatrix}
1 & 2 \\
3 & 4
\end{bmatrix}
\begin{bmatrix}
1 & 0 \\
0 & 0
\end{bmatrix}
\begin{bmatrix}
4 & 3 \\
2 & 1
\end{bmatrix}

= 
\begin{bmatrix}
4 & 3 \\
12 & 9
\end{bmatrix}

$$

Isso é equivalente a simplemente remover esse valor de $\Sigma$, e remover as linhas e colunas de $U$ e $V^T$ que seriam multiplicadas por ele. Daí, ficamos com:
$$
\begin{bmatrix}
1  \\
3 
\end{bmatrix}
\begin{bmatrix}
1 
\end{bmatrix}
\begin{bmatrix}
4 & 3 \\
\end{bmatrix}
= 
\begin{bmatrix}
4 & 3 \\
12 & 9
\end{bmatrix}
$$

A vantagem de remover linhas e colunas diretamente é que passamos a precisar de menos espaço para armazenar nossos dados!

---

Analise o código abaixo. Ele é bastante parecido com o código acima, Altere a função `comprimir` para que ela realize a remoção de elementos que foi mostrada acima. Teste seu código!!!

a) Como o valor de $K$ influencia a compressão da imagem e a qualidade da imagem final?

b) Qual é o menor valor de $K$ para o qual a imagem não perde qualidade?

c) Qual é o maior valor de $K$ que permite reduzir o tamanho ocupado pela imagem?

In [ ]:
from scipy.linalg import svd, diagsvd
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np

# Altere esta função!
def comprimir (u, s, vt, K):
    """Remove elementos de u, s e vt deixando somente K componentes restantes
    """ 
    u_ = u[:,K:K+1]
    s_ = s[K:K+1]
    vt_ = vt[K:K+1,:]
    return u_, s_, vt_

A = mpimg.imread("rabbit.jpeg")

u0, s0, vt0 = svd(A[:,:,0])
u1, s1, vt1 = svd(A[:,:,1])
u2, s2, vt2 = svd(A[:,:,2])

n_bytes_inicial = A.nbytes
print("Tamanho inicial da imagem (bytes): ", n_bytes_inicial)

K = 4

u0, s0, vt0 = comprimir(u0, s0, vt0, K)
u1, s1, vt1 = comprimir(u1, s1, vt1, K)
u2, s2, vt2 = comprimir(u2, s2, vt2, K)

sigma0 = diagsvd(s0, u0.shape[1], vt0.shape[0])
sigma1 = diagsvd(s1, u1.shape[1], vt1.shape[0])
sigma2 = diagsvd(s2, u2.shape[1], vt2.shape[0])

n_bytes_final = u0.nbytes + u1.nbytes + u2.nbytes + s0.nbytes + s1.nbytes + s2.nbytes + vt0.nbytes + vt1.nbytes + vt2.nbytes
print("Tamanho da representação comprimida: ", n_bytes_final)
print("Compressão:", n_bytes_final / n_bytes_inicial)

B = np.zeros_like(A)
B[:,:,0] = u0 @ sigma0 @ vt0
B[:,:,1] = u1 @ sigma1 @ vt1
B[:,:,2] = u2 @ sigma2 @ vt2

plt.subplot(1,2,1)
plt.title('Imagem original')
plt.imshow(A, cmap='gray')
plt.subplot(1,2,2)
plt.title('Imagem comprimida')
plt.imshow(B, cmap='gray')
plt.show()

# 